In [ ]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import warnings
warnings.simplefilter("ignore")

In [ ]:
if "test_chars_rank_imputed" not in os.listdir():
    os.mkdir("test_chars_rank_imputed")

if "test_chars_raw_no_impute" not in os.listdir():
    os.mkdir("test_chars_raw_no_impute")

if "test_chars_raw_no_impute_decile_bar" not in os.listdir():
    os.mkdir("test_chars_raw_no_impute_decile_bar")

if "test_chars_raw_no_impute_decile_only_mean" not in os.listdir():
    os.mkdir("test_chars_raw_no_impute_decile_only_mean")

if "test_chars_raw_no_impute_decile_box_plot" not in os.listdir():
    os.mkdir("test_chars_raw_no_impute_decile_box_plot")

In [ ]:
# char_list = ['maxret', 'grltnoa', 'pchcurrat', 'abr', 'pm', 'pchsale_pchinvt',
#        'salecash', 'herf', 'cashdebt', 'sp', 'std_turn', 'rdm', 'roic',
#        'chinv', 'adm', 'acc', 'me', 'tang', 'pchgm_pchsale', 'absacc',
#        'pchsale_pchrect', 'rvar_ff3', 'tb', 'chempia', 'mom1m', 'invest',
#        'roe', 'convind', 'std_dolvol', 'op', 'cfp_ia', 'seas1a',
#        'secured', 'chpmia', 'pscore', 'quick', 'baspread', 'mom6m',
#        'pctacc', 'rd', 'dy', 'realestate', 'chatoia', 're', 'agr', 'divi',
#        'ni', 'rna', 'nincr', 'chcsho', 'indmom', 'egr', 'salerec', 'roa',
#        'cinvest', 'pchdepr', 'gma', 'securedind', 'currat', 'divo', 'cfp',
#        'ato', 'cashpr', 'ill', 'beta', 'roavol', 'turn', 'grcapx',
#        'pchquick', 'ep', 'me_ia', 'zerotrade', 'rvar_mean', 'depr',
#        'pchsaleinv', 'hire', 'stdcf', 'chtx', 'rd_sale', 'saleinv',
#        'rsup', 'rvar_capm', 'bm_ia', 'mom36m', 'pchsale_pchxsga', 'alm',
#        'mom12m', 'pchcapx_ia', 'sgr', 'sue', 'lev', 'age', 'mom60m', 'bm',
#        'sin', 'cash', 'dolvol', 'lgr', 'noa', 'stdacc', 'chmom']
char_list = ['cinvest', 'rvar_capm', 'mom36m', 'zerotrade', 'rna', 'chtx', 'pm',
       'depr', 'cash', 'me_ia', 'adm', 'bm_ia', 'cashdebt',
       'mom12m', 'baspread', 'sp', 'mom1m', 'rvar_ff3', 'ep', 'dy', 're',
       'nincr', 'rdm', 'rsup', 'lgr', 'chpmia', 'std_dolvol', 'rd_sale',
       'beta', 'rvar_mean', 'mom60m', 'chcsho', 'roa', 'acc', 'ato',
       'sue', 'op', 'alm', 'noa', 'mom6m', 'me', 'cfp', 'pscore',
       'seas1a', 'roe', 'maxret', 'std_turn', 'sgr', 'grltnoa', 'gma',
       'ni', 'dolvol', 'bm', 'pctacc', 'herf', 'lev', 'agr', 'ill', 'abr',
       'hire', 'turn']
char_list = sorted(char_list)
rank_columns = [f"rank_{col}" for col in char_list]

In [ ]:
df_raw_no_impute = pd.read_feather("chars/chars_raw_no_impute.feather")
df_raw_no_impute['date'] = pd.to_datetime(df_raw_no_impute['date']).dt.strftime('%Y-%m-%d')
print(df_raw_no_impute.shape)

df_raw_imputed = pd.read_feather("chars/chars_raw_imputed.feather")
df_raw_imputed['date'] = pd.to_datetime(df_raw_imputed['date']).dt.strftime('%Y-%m-%d')
print(df_raw_imputed.shape)

df_rank_no_impute = pd.read_feather("chars/chars_rank_no_impute.feather")
df_rank_no_impute['date'] = pd.to_datetime(df_rank_no_impute['date']).dt.strftime('%Y-%m-%d')
print(df_rank_no_impute.shape)

df_rank_imputed = pd.read_feather("chars/chars_rank_imputed.feather")
df_rank_imputed['date'] = pd.to_datetime(df_rank_imputed['date']).dt.strftime('%Y-%m-%d')
print(df_rank_imputed.shape)

#### Filter out 61 chars version

In [ ]:
df_raw_no_impute = df_raw_no_impute[['gvkey', 'permno', 'sic', 'ret', 'exchcd', 'shrcd', 'ticker', 'conm', 'comnam', 'date', 'prc', 'shrout'] + char_list]
df_raw_no_impute.to_feather("chars60_raw_no_impute.feather")

In [ ]:
df_raw_imputed = df_raw_imputed[['gvkey', 'permno', 'sic', 'ret', 'exchcd', 'shrcd', 'ticker', 'conm', 'comnam', 'date','ffi49', 'prc', 'shrout'] + char_list]
df_raw_imputed.to_feather("chars60_raw_imputed.feather")

In [ ]:
df_rank_no_impute = df_rank_no_impute[['gvkey', 'permno', 'sic', 'ret', 'exchcd', 'shrcd', 'date', 'lag_me', 'ticker', 'conm', 'comnam', 'prc', 'shrout', 'log_me'] + rank_columns]
df_rank_no_impute.to_feather("chars60_rank_no_impute.feather")

In [ ]:
df_rank_imputed = df_rank_imputed[['gvkey', 'permno', 'sic', 'ret', 'exchcd', 'shrcd', 'date', 'lag_me', 'ticker', 'conm', 'comnam', 'prc', 'shrout', 'log_me'] + rank_columns]
df_rank_imputed.to_feather("chars60_rank_imputed.feather")

In [ ]:
df_raw_no_impute[df_raw_no_impute['date'] >= '2014-05-01']['bm'].describe()

#### Start to check our data

In [ ]:
all_dates = sorted(list(df_rank_imputed['date'].unique()))

##### Draw rank imputed chars histogram

In [ ]:
subplot_x = math.ceil(len(char_list) / 5)

def plot_for_date(this_date):
    select_char = df_rank_imputed[df_rank_imputed['date'] == this_date]

    # plot all the char
    fig, axs = plt.subplots(subplot_x, 5, figsize=(20, int(3*subplot_x)))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)

    for i, column in enumerate(rank_columns):
        ax = axs[i // 5, i % 5]
        ax.hist(select_char[column], bins=20, range=(-1, 1), edgecolor='black')
        ax.set_title(f'char: {column}')
        ax.set_xlabel('Values')
        ax.set_ylabel('Frequency')
    plt.savefig(f"test_chars_rank_imputed/char_plot_{str(this_date)[0:4]}_{str(this_date)[5:7]}.png")
    plt.show()
    plt.close()

Parallel(n_jobs=50)(delayed(plot_for_date)(this_date) for this_date in all_dates)

##### Draw raw no-impute histogram

In [ ]:
subplot_x = math.ceil(len(char_list) / 5)

def plot_for_date(this_date):
    select_char = df_raw_no_impute[df_raw_no_impute['date'] == this_date]

    # plot all the char
    fig, axs = plt.subplots(subplot_x, 5, figsize=(20, int(3*subplot_x)))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)

    for i, column in enumerate(char_list):
        non_na_values = select_char[column].dropna()
        if not non_na_values.empty:
            min_value = non_na_values.min()
            max_value = non_na_values.max()
        else:
            min_value = -1
            max_value = 1
        
        ax = axs[i // 5, i % 5]
        ax.hist(select_char[column], bins=20, range=(min_value, max_value), edgecolor='black')
        ax.set_title(f'char: {column}')
        ax.set_xlabel('Values')
        ax.set_ylabel('Frequency')
    plt.savefig(f"test_chars_raw_no_impute/char_plot_{str(this_date)[0:4]}_{str(this_date)[5:7]}.png")
    plt.show()
    plt.close()

Parallel(n_jobs=50)(delayed(plot_for_date)(this_date) for this_date in all_dates)

##### Draw raw no-impute decile barplot

In [ ]:
# df_temp = df_raw_no_impute[df_raw_no_impute['date'] == '1950-11-30'].reset_index(drop=True)

# char = 'hire'
# this_char = df_temp[[char]].dropna().reset_index(drop=True)
# print(f"Char {char}: {this_char.shape}")

# if len(this_char) < 10:
#     print("No data for this char")
# else:
#     this_char['decile'] = pd.qcut(this_char[char], 10, labels=False, duplicates='drop')
#     if len(this_char['decile'].unique()) == 1:
#         this_char['decile'] = 1
#     decile_stats = this_char.groupby('decile')[char].agg(['mean', 'min', 'max']).reset_index()

#     print(decile_stats)

In [ ]:
subplot_x = math.ceil(len(char_list) / 3)

def plot_for_date(this_date):
    select_char = df_raw_no_impute[df_raw_no_impute['date'] == this_date].reset_index(drop=True)

    # plot all the char
    fig, axs = plt.subplots(subplot_x, 3, figsize=(20, int(4*subplot_x)))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)

    for i, column in enumerate(char_list):
        non_na_values = select_char[[column]].dropna().reset_index(drop=True)
        if len(non_na_values) >= 10:
            # 将数据分为10个decile，如果唯一值数量少于10，则分为唯一值数量的分位数
            num_bins = 10
            non_na_values['decile'] = pd.qcut(non_na_values[column], num_bins, labels=False, duplicates='drop') + 1
            if len(non_na_values['decile'].unique()) == 1:
                non_na_values['decile'] = 1

            # 计算每个decile的均值、最小值和最大值
            decile_stats = non_na_values.groupby('decile')[column].agg(['mean', 'min', 'max']).reset_index()
            print(f"date: {this_date}, char: {column}, decile: {decile_stats['decile'].unique()}")

            # 绘制散点图
            ax = axs[i // 3, i % 3]
            ax.scatter(decile_stats['decile'], decile_stats['mean'], label='Mean', color='blue', marker='o')
            ax.scatter(decile_stats['decile'], decile_stats['min'], label='Min', color='red', marker='o')
            ax.scatter(decile_stats['decile'], decile_stats['max'], label='Max', color='green', marker='o')
            ax.set_title(f'char: {column}')
            ax.set_xlabel('Decile')
            ax.set_ylabel('Value')
            max_decile = int(decile_stats['decile'].max())
            ax.set_xticks(range(1, max_decile + 1))
            ax.set_xticklabels(range(1, max_decile + 1))
            ax.legend()
        else:
            continue

    plt.savefig(f"test_chars_raw_no_impute_decile_bar/char_plot_{str(this_date)[0:4]}_{str(this_date)[5:7]}.png")
    plt.show()
    plt.close()

Parallel(n_jobs=50)(delayed(plot_for_date)(this_date) for this_date in all_dates)

In [ ]:
subplot_x = math.ceil(len(char_list) / 3)

def plot_for_date(this_date):
    select_char = df_raw_no_impute[df_raw_no_impute['date'] == this_date].reset_index(drop=True)

    # plot all the char
    fig, axs = plt.subplots(subplot_x, 3, figsize=(20, int(4*subplot_x)))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)

    for i, column in enumerate(char_list):
        non_na_values = select_char[[column]].dropna().reset_index(drop=True)
        if len(non_na_values) >= 10:
            # 将数据分为10个decile，如果唯一值数量少于10，则分为唯一值数量的分位数
            num_bins = 10
            non_na_values['decile'] = pd.qcut(non_na_values[column], num_bins, labels=False, duplicates='drop') + 1
            if len(non_na_values['decile'].unique()) == 1:
                non_na_values['decile'] = 1

            # 计算每个decile的均值、最小值和最大值
            decile_stats = non_na_values.groupby('decile')[column].agg(['mean', 'min', 'max']).reset_index()
            print(f"date: {this_date}, char: {column}, decile: {decile_stats['decile'].unique()}")

            # 绘制散点图
            ax = axs[i // 3, i % 3]
            ax.scatter(decile_stats['decile'], decile_stats['mean'], label='Mean', color='red', marker='o')
            ax.set_title(f'char: {column}')
            ax.set_xlabel('Decile')
            ax.set_ylabel('Value')
            max_decile = int(decile_stats['decile'].max())
            ax.set_xticks(range(1, max_decile + 1))
            ax.set_xticklabels(range(1, max_decile + 1))
            ax.legend()
        else:
            continue

    plt.savefig(f"test_chars_raw_no_impute_decile_only_mean/char_plot_{str(this_date)[0:4]}_{str(this_date)[5:7]}.png")
    plt.show()
    plt.close()

Parallel(n_jobs=50)(delayed(plot_for_date)(this_date) for this_date in all_dates)

##### Draw raw no-impute box plot

In [ ]:
subplot_x = math.ceil(len(char_list) / 3)

def plot_for_date(this_date):
    select_char = df_raw_no_impute[df_raw_no_impute['date'] == this_date].reset_index(drop=True)

    # plot all the char
    fig, axs = plt.subplots(subplot_x, 3, figsize=(20, int(4*subplot_x)))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)

    for i, column in enumerate(char_list):
        non_na_values = select_char[[column]].dropna().reset_index(drop=True)
        if len(non_na_values) >= 10:
            # 将数据分为10个decile，如果唯一值数量少于10，则分为唯一值数量的分位数
            num_bins = 10
            non_na_values['decile'] = pd.qcut(non_na_values[column], num_bins, labels=False, duplicates='drop') + 1
            if len(non_na_values['decile'].unique()) == 1:
                non_na_values['decile'] = 1

            # 计算每个decile的均值、最小值和最大值
            decile_stats = non_na_values.groupby('decile')[column].agg(['mean', 'min', 'max']).reset_index()
            print(f"date: {this_date}, char: {column}, decile: {decile_stats['decile'].unique()}")

            # 绘制箱线图
            ax = axs[i // 3, i % 3]
            non_na_values.boxplot(column=column, by='decile', ax=ax)
            ax.set_title(f'char: {column}')
            ax.set_xlabel('Decile')
            ax.set_ylabel('Value')
            max_decile = int(decile_stats['decile'].max())
            ax.set_xticks(range(1, max_decile + 1))
            ax.set_xticklabels(range(1, max_decile + 1))
        else:
            continue

    plt.suptitle(f'Boxplots for {this_date}')
    plt.savefig(f"test_chars_raw_no_impute_decile_box_plot/char_plot_{str(this_date)[0:4]}_{str(this_date)[5:7]}.png")
    plt.show()
    plt.close()

Parallel(n_jobs=50)(delayed(plot_for_date)(this_date) for this_date in all_dates)

In [ ]:
subplot_x = math.ceil(len(char_list) / 3)

this_date = '2024-12-31'

select_char = df_raw_no_impute[df_raw_no_impute['date'] == this_date].reset_index(drop=True)

# plot all the char
fig, axs = plt.subplots(subplot_x, 3, figsize=(20, int(4*subplot_x)))
fig.subplots_adjust(hspace=0.5, wspace=0.5)

for i, column in enumerate(char_list):
    non_na_values = select_char[[column]].dropna().reset_index(drop=True)
    if len(non_na_values) >= 10:
        # 将数据分为10个decile，如果唯一值数量少于10，则分为唯一值数量的分位数
        num_bins = 10
        non_na_values['decile'] = pd.qcut(non_na_values[column], num_bins, labels=False, duplicates='drop') + 1
        if len(non_na_values['decile'].unique()) == 1:
            non_na_values['decile'] = 1

        # 计算每个decile的均值、最小值和最大值
        non_na_values = non_na_values[non_na_values['decile'] != 1].reset_index(drop=True)
        non_na_values = non_na_values[non_na_values['decile'] != 10].reset_index(drop=True)
        decile_stats = non_na_values.groupby('decile')[column].agg(['mean', 'min', 'max']).reset_index()
        # print(f"date: {this_date}, char: {column}, decile: {decile_stats['decile'].unique()}")

        # 绘制箱线图
        ax = axs[i // 3, i % 3]
        non_na_values.boxplot(column=column, by='decile', ax=ax)
        ax.set_title(f'char: {column}')
        ax.set_xlabel('Decile')
        ax.set_ylabel('Value')
        max_decile = int(decile_stats['decile'].max())
        ax.set_xticks(range(1, max_decile))
        ax.set_xticklabels(range(2, max_decile+1))
    else:
        continue

plt.suptitle(f'Boxplots for {this_date}')
# plt.savefig(f"test_chars_raw_no_impute_decile_box_plot/char_plot_{str(this_date)[0:4]}_{str(this_date)[5:7]}.png")
plt.show()
plt.close()